
# **AI LAB — Hafta 5: Uzman Sistem Uygulaması (Experta)**
**Klasör:** `C:\Users\The Coder Farmer\Desktop\AI LAB`  
**Dosya:** `AI_LAB_05_Uzman_Sistem.ipynb`

Bu dosya, PDF'teki *Uygulama 7: Uzman Sistem Uygulaması* kapsamında istenen kuralları (diş ağrısı senaryosu) **Experta** (Python kural tabanlı sistem kütüphanesi) ile gerçekliyor.  
Aşağıdaki hücreleri sırayla çalıştır: kurulum → kuralların tanımı → testler → etkileşimli kullanım.



## 1) Kurulum
Aşağıdaki hücre, gerekli kütüphaneyi yükler. (İlk çalıştırmada internet gerekir. Yerel kuruluysa hızlıca geçecektir.)


In [1]:
# Python 3.10+ için experta uyumluluk düzeltmesi
import collections, collections.abc
for attr in ("Mapping", "MutableMapping", "Sequence"):
    if not hasattr(collections, attr):
        setattr(collections, attr, getattr(collections.abc, attr))


In [2]:


try:
    import experta  # type: ignore
    print("experta zaten yüklü.")
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "experta"])
    import experta  # yeniden dene
    print("experta yüklendi.")


experta zaten yüklü.



## 2) Kısa Arkaplan
**Uzman sistem**: Bilgi tabanı (kurallar) + Çıkarım motoru (ileri zincirleme) ile **koşul → eylem** mantığıyla karar üretir.  
Bu örnekte **girdi**, ağız/diş durumuna ilişkin belirtiler; **çıktı**, uygulanacak işlem/öneridir.



### 2.1) Kurallar (Diş Ağrısı Senaryosu)
Aşağıdaki kurallar PDF'teki *Değerlendirme Soruları* bölümünden alınmıştır (özet):

1. **Diş fırçalarken diş eti kanaması** → *diş hastalığı vardır ve diş hekimine başvur.*  
2. **Uzun süreli diş eti kanaması** → *diş eti çekilmesi vardır ve diş hekimine başvur.*  
3. **Diş eti çekilmesi + diş kökü görünmesi** → *dolgu yaptır.*  
4. **Yiyecek/İçecek kaynaklı renk değişimi** → *dişleri temizle.*  
5. **Yeni diş çıkarken morarma** → *diş hekimine başvur.*  
6. **Ağrı yapmayan çürük** → *dolgu yaptır.*  
7. **Çürük ileri derecede** → *kanal tedavisi ve dolgu yaptır.*



## 3) Uygulama: Experta ile Kural Tabanlı Sistem
Aşağıda gerçekler (**Facts**) ve kurallar (**Rules**) tanımlanır.  
Kullanım: `run_engine(**belirti_sözlüğü)` fonksiyonuna belirtileri ver, sistem uygun eylemleri üretir.


In [3]:

from experta import Fact, Rule, KnowledgeEngine, AND

class Dental(Fact):
    """Diş/dişeti ile ilgili belirtiler."""
    pass

class DentalExpert(KnowledgeEngine):
    def __init__(self):
        super().__init__()
        self.actions = []  # üretilen önerileri burada toplayacağız

    # 1) Diş fırçalarken diş eti kanaması → hekime başvur
    @Rule(Dental(bleeding_on_brush=True))
    def rule_bleeding_brush(self):
        self.actions.append("Diş hastalığı olabilir → diş hekimine başvur.")

    # 2) Uzun süreli kanama → diş eti çekilmesi olabilir → hekime başvur
    @Rule(Dental(long_bleeding=True))
    def rule_long_bleeding(self):
        self.actions.append("Diş eti çekilmesi belirtisi → diş hekimine başvur.")

    # 3) Diş eti çekilmesi + kök görünmesi → dolgu
    @Rule(AND(Dental(gum_recession=True), Dental(root_visible=True)))
    def rule_recession_root(self):
        self.actions.append("Diş eti çekilmesi ve kök görünümü → dolgu yaptır.")

    # 4) Yiyecek/İçecek kaynaklı renk değişimi → temizlik
    @Rule(Dental(stain_from_food=True))
    def rule_stain(self):
        self.actions.append("Renk değişimi var → diş taşı/lekeler için temizlik yaptır.")

    # 5) Yeni diş çıkarken morarma → hekime başvur
    @Rule(Dental(bruising_new_tooth=True))
    def rule_bruising(self):
        self.actions.append("Yeni diş çıkarken morarma → diş hekimine başvur.")

    # 6) Ağrı yapmayan çürük → dolgu
    @Rule(Dental(painless_caries=True))
    def rule_painless_caries(self):
        self.actions.append("Ağrı yapmayan çürük → dolgu yaptır.")

    # 7) İleri derecede çürük → kanal + dolgu
    @Rule(Dental(severe_caries=True))
    def rule_severe_caries(self):
        self.actions.append("İleri derecede çürük → kanal tedavisi + dolgu yaptır.")

def run_engine(**symptoms):
    """Belirti sözlüğünü (bool) alır, kuralları çalıştırır, önerileri döndürür."""
    engine = DentalExpert()
    engine.reset()
    engine.declare(Dental(**symptoms))
    engine.run()
    # Eğer hiçbir kural tetiklenmediyse, kullanıcıyı genel muayeneye yönlendir.
    return engine.actions or ["Belirtiler kurallarla eşleşmedi → genel muayene için diş hekimine görün."]



## 4) Hızlı Testler
Aşağıdaki örnekler her kuralı en az bir kez tetikler.


In [4]:

tests = [
    dict(bleeding_on_brush=True),
    dict(long_bleeding=True),
    dict(gum_recession=True, root_visible=True),
    dict(stain_from_food=True),
    dict(bruising_new_tooth=True),
    dict(painless_caries=True),
    dict(severe_caries=True),
    dict()  # kuralsız
]

for i, t in enumerate(tests, 1):
    print(f"Test {i} -> input: {t}")
    for rec in run_engine(**t):
        print("  •", rec)
    print()


Test 1 -> input: {'bleeding_on_brush': True}
  • Diş hastalığı olabilir → diş hekimine başvur.

Test 2 -> input: {'long_bleeding': True}
  • Diş eti çekilmesi belirtisi → diş hekimine başvur.

Test 3 -> input: {'gum_recession': True, 'root_visible': True}
  • Diş eti çekilmesi ve kök görünümü → dolgu yaptır.

Test 4 -> input: {'stain_from_food': True}
  • Renk değişimi var → diş taşı/lekeler için temizlik yaptır.

Test 5 -> input: {'bruising_new_tooth': True}
  • Yeni diş çıkarken morarma → diş hekimine başvur.

Test 6 -> input: {'painless_caries': True}
  • Ağrı yapmayan çürük → dolgu yaptır.

Test 7 -> input: {'severe_caries': True}
  • İleri derecede çürük → kanal tedavisi + dolgu yaptır.

Test 8 -> input: {}
  • Belirtiler kurallarla eşleşmedi → genel muayene için diş hekimine görün.




## 5) Etkileşimli Kullanım
Aşağıdaki hücre, konsoldan **E/H** (Evet/Hayır) girişleri alarak belirtileri oluşturur.


In [12]:

def ask_bool(prompt):
    ans = input(prompt + " (E/H): ").strip().lower()
    return ans.startswith('e')

symptoms = {
    "bleeding_on_brush": ask_bool("Diş fırçalarken diş eti kanaması var mı?"),
    "long_bleeding": ask_bool("Uzun süreli diş eti kanaması var mı?"),
    "gum_recession": ask_bool("Diş eti çekilmesi var mı?"),
    "root_visible": ask_bool("Diş kökü görünüyor mu?"),
    "stain_from_food": ask_bool("Yiyecek/içecek kaynaklı renk değişimi var mı?"),
    "bruising_new_tooth": ask_bool("Yeni diş çıkarken morarma var mı?"),
    "painless_caries": ask_bool("Ağrı yapmayan çürük var mı?"),
    "severe_caries": ask_bool("Çürük ileri derecede mi?"),
}

print("\nÖNERİLER:")
for rec in run_engine(**symptoms):
    print("•", rec)



ÖNERİLER:
• Yeni diş çıkarken morarma → diş hekimine başvur.
• Diş eti çekilmesi belirtisi → diş hekimine başvur.
• Diş eti çekilmesi ve kök görünümü → dolgu yaptır.
• Diş hastalığı olabilir → diş hekimine başvur.
• İleri derecede çürük → kanal tedavisi + dolgu yaptır.
• Ağrı yapmayan çürük → dolgu yaptır.
• Renk değişimi var → diş taşı/lekeler için temizlik yaptır.



---
### Notlar
- **Kural çatışması** durumunda (birden çok öneri) sistem hepsini listeler; klinik uygulamada **önceliklendirme** yapılabilir.
- İstersen aşağıdaki sözlüğe **ek belirtiler** ekleyip yeni kurallar tanımlayabilirsin.
- Uygulama, ileri zincirleme (forward-chaining) kullanan **Experta** `KnowledgeEngine` yapısını temel alır.
